In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse
import warnings
warnings.simplefilter('ignore')

In [ ]:
def fix_link(u, to="vi"):
    id = u.find(".html")
    if to == "vi":
        if u[int(id - 5):int(id)] != "en-gb" and u[int(id - 2):int(id)] != "vi":
            return u.replace(".html", ".vi.html")
        else:
            return u.replace("en-gb.html", "vi.html")
    elif to == "en":
        if u[int(id - 2):int(id)] == "vi":
            return u.replace("vi.html", "en-gb.html")
        

In [ ]:
def get_full_url_path(url):
    parsed_url = urlparse(url)
    scheme = parsed_url.scheme
    netloc = parsed_url.netloc
    path = parsed_url.path
    full_path = f"{scheme}://{netloc}{path}"
    return full_path

In [ ]:
def extract_hotel_links(url, processed_links):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    property_cards = soup.find_all("div", {"data-testid": "property-card"}) 
    print(len(property_cards))    
    properties = []
    for property in property_cards:
        new_property = {}
        try:
            link = property.find('a', {'data-testid': 'title-link'}).get('href')
            linked = fix_link(link)
            new_property['link'] = get_full_url_path(linked)
        except:
            new_property['link'] = np.nan
        # Skip if duplicate
        if new_property['link'] in processed_links:
            continue
        try:
            new_property['hotel_name'] = property.find('div', {'data-testid': 'title'}).text
        except:
            new_property['hotel_name'] = np.nan
        try:
            review_score, review_count = property.find('div', {'data-testid': 'review-score'})
            new_property['review_score'] = review_score.text.split(" ")[2]
            
            new_property['review_count'] = review_count.text.split(" ")[-3]
        except:
            new_property['review_score'] = np.nan
            new_property['review_count'] = np.nan
        try:
            new_property['price'] = property.find('span', {'data-testid': 'price-and-discounted-price'}).text
        except:
            new_property['price'] = np.nan
        try:
            new_property['address'] = property.find('span', {'data-testid': 'address'}).text
        except:
            new_property['address'] = np.nan
        try:
            new_property['image'] = property.find('img', {'data-testid': 'image'}).get('src')
        except:
            new_property['image'] = np.nan
        properties.append(new_property)
        processed_links.append(new_property['link'])
    return properties, processed_links

In [ ]:
def crawling_from_booking(filter_star):
    url = f"https://www.booking.com/searchresults.vi.html?aid=304142&ss=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&ssne=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&ssne_untouched=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&lang=vi&dest_id=-3730078&dest_type=city&checkin=2024-04-10&checkout=2024-04-11&group_adults=2&no_rooms=1&group_children=0&nflt=class%3D{filter_star}&order=price&offset="
    # url = f"https://www.booking.com/searchresults.vi.html?laid=397594&ss=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&ssne=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&ssne_untouched=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&lang=vi&dest_id=-3730078&dest_type=city&checkin=2024-04-02&checkout=2024-04-03&group_adults=2&no_rooms=1&group_children=0&nflt=class%3D{filter_star}&order=price&offset="
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(options=options)
    driver.get(url+"25")
    processed_links = []
    properties = []
    try:
        pages = driver.find_elements(By.XPATH, '//div[@data-testid="pagination"]//li')
        last_page = int(pages[-1].text)
    except:
        last_page = 1
    for current_page in range(last_page):
            driver.get(url+str(current_page*25))
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2) # Đợi để trang web tải thêm kết quả
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
            time.sleep(5)
            print(driver.current_url)
            new_properties, processed_links = extract_hotel_links(driver.current_url, processed_links)
            properties = properties + new_properties
    df = pd.DataFrame(properties)
    df.to_excel("Crawling_results.xlsx")
    return properties

In [ ]:
def crawling_from_booking_2(filter_star):
    url = f"https://www.booking.com/searchresults.vi.html?aid=304142&ss=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&ssne=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&ssne_untouched=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&highlighted_hotels=10322077&efdco=1&lang=vi&dest_id=-3730078&dest_type=city&checkin=2024-04-10&checkout=2024-04-11&group_adults=2&no_rooms=1&group_children=0&order=class&nflt=class%3D{filter_star}"
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    processed_links = []
    properties = []
    # Lấy chiều cao ban đầu của trang
    for i in range(10):
        driver.execute_script("window.scrollBy(0, 2000);")
        time.sleep(1)  # Đợi để trang web tải thêm dữ liệu
    # Lấy tất cả kết quả đã tải
    # ád
    new_properties, processed_links = extract_hotel_links(driver.current_url, processed_links)
    properties.extend(new_properties)

    df = pd.DataFrame(properties)
    df.to_excel(f"Crawling_{filter_star}.xlsx")
    return properties

In [ ]:
import re
def fix_facilities(facilities):
    # Sử dụng regex để tìm các từ khóa bắt đầu bằng chữ hoa
    pattern = r'\b[A-Z]\w*'
    facilities_list = re.findall(pattern, facilities)
    if "Hồ" in facilities or "hồ" in facilities:
        facilities_list[0] = "Hồ bơi ngoài trời"
    else:
        facilities_list[0] = "2 hồ bơi"
    # Xử lý các trường hợp đặc biệt
    for i, facility in enumerate(facilities_list):
        if facility == 'WiFi':
            facilities_list[i] = 'WiFi miễn phí'
        elif facility == 'Sân':
            facilities_list[i] = 'Sân thượng / hiên'
        elif facility == 'Bữa':
            if 'tuyệt' in facilities:
                facilities_list[i] = 'Bữa sáng tuyệt hảo'
            else:
                facilities_list[i] = 'Bữa sáng rất tốt'
        elif facility == 'Xe':
            facilities_list[i] = 'Xe đưa đón sân bay'
        elif facility == 'Pho':
            if 'Phòng gia đình' in facilities_list:
                facilities_list[i] = 'Phòng không hút thuốc'
            else:
                facilities_list[i] = 'Phòng gia đình'
        elif facility == 'Trung':
            if facilities_list[i+1] == 'Spa' and i + 1 < len(facilities_list):
                    facilities_list[i] = 'Trung tâm Spa & chăm sóc sức khoẻ'
                    if i + 1 < len(facilities_list):
                        facilities_list.pop(i + 1)
            else:
                facilities_list[i] = 'Trung tâm thể dục'
        elif facility == 'Chỗ' or facility == 'Chô':
            if '(trong khuôn viên)' in facilities:
                facilities_list[i] = 'Chỗ đậu xe (trong khuôn viên)'
            else:
                facilities_list[i] = 'Chỗ đỗ xe miễn phí'
        elif facility == 'Quầy':
            facilities_list[i] = 'Quầy bar'
        elif facility == 'Giáp':
            facilities_list[i] = 'Giáp biển'
        elif facility == 'Khu':
            facilities_list[i] = 'Khu vực bãi tắm riêng'
        elif facility == 'Lê' or facility == 'Lễ':
            facilities_list[i] = 'Lễ tân 24h' 
        elif facility == 'Nha' or facility == 'Nhà':
            facilities_list[i] = 'Nhà hàng'
        elif facility == 'Dịch':
            facilities_list[i] = 'Dịch vụ đưa đón sân bay (miễn phí)'
        elif facility == 'Di':
            facilities_list[i] = 'Dịch vụ phòng'
        elif facility == 'Điều':
            facilities_list[i] = 'Điều hòa nhiệt độ' 
        elif facility == 'Máy':
            facilities_list[i] = 'Máy pha trà/cà phê trong tất cả các phòng'
    return facilities_list

In [ ]:
def fix_subscore(review_subscore):
    result = {}
    for item in review_subscore:
        match = re.match(r'(.*?)(\d+,?\d+)', item)
        if match:
            key, value = match.groups()
            value = value.replace(',', '.')
            result[key.strip()] = float(value)
    return result

In [ ]:
def fix_address(address):
    result = re.split(r',', address)[0]
    return result

In [ ]:
def fix_scores(scores):
    result = scores.split(" ")[-1]
    return result

In [ ]:
def crawling_from_links(data,filter_star):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    properties = data.copy()
    properties['facilities'] =np.full(len(properties), np.nan)
    properties['subscore'] = np.full(len(properties), np.nan)
    for item in range(len(properties)):
        properties['link'][item] = get_full_url_path(properties['link'][item])
        properties['address'][item] = fix_address(properties['address'][item])
        try:
            properties['review_score'][item] = fix_scores(properties['review_score'][item])
        except:
            properties['review_score'][item] = np.nan
        resp = requests.get(properties['link'][item], headers=headers)
        soup = BeautifulSoup(resp.text, 'html.parser')
        try:
            facilities = soup.find('div',{'data-testid':'property-most-popular-facilities-wrapper'}).text
            properties['facilities'][item] = fix_facilities(facilities)
        except:
            properties['facilities'][item] = np.nan
        try:
            subscore = soup.find_all('div',{'data-testid':'review-subscore'})
            review_subscore = [sub.get_text(strip=True) for sub in subscore]
            properties['subscore'][item] = fix_subscore(review_subscore)
        except:
            properties['subscore'][item] = np.nan
    properties.to_excel(f"new_crawling_{filter_star}.xlsx")
    return properties

In [ ]:
def load_properties(filter_star):
    with open(f'crawling_{filter_star}.xlsx', 'rb') as file:
        properties = pd.read_excel(file)
    return properties

In [ ]:
properties=load_properties(5)

In [ ]:
properties['link'][0]

In [ ]:
# properties = crawling_from_booking_2("5")

In [ ]:
for i in range(5):
    properties=load_properties(i)
    test = crawling_from_links(properties,i)
    print(test['review_score'])

In [ ]:
properties['review_score']

In [ ]:

def convert_price(price_str, convert_missing=False):
    if pd.isna(price_str) and not convert_missing:
        return price_str  # Giữ nguyên giá trị NaN
    else:
        price_str = str(price_str)
        price_str = re.sub(r'[^\d,]+', '', price_str)  # Loại bỏ tất cả ký tự không phải số và dấu phẩy
        price_str = price_str.replace(',', '')
        return int(price_str)

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
fulldata = pd.DataFrame()  # Khởi tạo DataFrame rỗng

for i in range(6):
    data = pd.read_excel(f'cleaning_data_{i}.xlsx')
    data['label'] = i
    fulldata = pd.concat([fulldata, data], ignore_index=True)
fulldata['subscore'] = fulldata['subscore'].apply(eval)
df = pd.DataFrame.from_records(fulldata['subscore'].tolist())
fulldata = pd.concat([fulldata, df], axis=1)
fulldata = fulldata.rename(columns={'Nhân viên phục vụ': 'staff_score', 'Tiện nghi': 'facility_score',
                                    'Sạch sẽ': 'clean_score', 'Thoải mái': 'Comfor_score',
                                    'Đáng giá tiền': 'price_score', 'Địa điểm': 'place_score',
                                    'WiFi miễn phí': 'wifi_score'})
fulldata['price'] = fulldata['price'].apply(convert_price, convert_dtype=False, convert_missing=False)
list_col = ['review_score','staff_score', 'facility_score', 'clean_score', 'Comfor_score', 'price_score', 'place_score', 'wifi_score']
for name in list_col:
# Create a scatter plot of Price vs. Review Count
    fulldata[name] = fulldata[name].astype(str).str.replace(',', '.')
    fulldata[name] = pd.to_numeric(fulldata[name], errors='coerce')
fulldata = fulldata.drop(columns=['subscore', 'facilities', 'Unnamed: 0'])
fulldata.to_csv('fulldata.csv')

In [11]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
fulldata = pd.DataFrame()  # Khởi tạo DataFrame rỗng

for i in range(6):
    data = pd.read_excel(f'cleaning_data_{i}.xlsx')
    data['label'] = i
    fulldata = pd.concat([fulldata, data], ignore_index=True)
fulldata['subscore'] = fulldata['subscore'].apply(eval)
df = pd.DataFrame.from_records(fulldata['subscore'].tolist())
fulldata = pd.concat([fulldata, df], axis=1)
fulldata = fulldata.rename(columns={'Nhân viên phục vụ': 'staff_score', 'Tiện nghi': 'facility_score',
                                    'Sạch sẽ': 'clean_score', 'Thoải mái': 'Comfor_score',
                                    'Đáng giá tiền': 'price_score', 'Địa điểm': 'place_score',
                                    'WiFi miễn phí': 'wifi_score'})
fulldata['price'] = fulldata['price'].apply(convert_price, convert_dtype=False, convert_missing=False)
list_col = ['review_score','staff_score', 'facility_score', 'clean_score', 'Comfor_score', 'price_score', 'place_score', 'wifi_score']
for name in list_col:
# Create a scatter plot of Price vs. Review Count
    fulldata[name] = fulldata[name].astype(str).str.replace(',', '.')
    fulldata[name] = pd.to_numeric(fulldata[name], errors='coerce')
fulldata = fulldata.drop(columns=['subscore', 'facilities', 'Unnamed: 0'])
fulldata.to_csv('fulldata.csv')

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).

In [4]:
import pandas as pd
df =  pd.read_excel('cleaning_data_3.xlsx')

In [3]:
subscore_df = df['subscore'].apply(eval).apply(pd.Series)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2440\676292070.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  subscore_df = df['subscore'].apply(eval).apply(pd.Series)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2440\676292070.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  subscore_df = df['subscore'].apply(eval).apply(pd.Series)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2440\676292070.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  subscore_df = df['subscore'].apply(eval).apply(pd.Series)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2440\676292070.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of

In [5]:
# Split the 'subscore' column into separate columns
subscore_df = df['subscore'].apply(eval).apply(pd.Series)

# Rename the columns
subscore_df.columns = ['Service staff', 'Facilities', 'Cleanliness', 'Comfort', 'Affordable', 'Location', 'Free WiFi']
# Concatenate the new columns with the original DataFrame
df = pd.concat([df, subscore_df], axis=1)

# Drop the original 'subscore' column
df.drop('subscore', axis=1, inplace=True)

# Specify the columns to drop
columns_to_drop = ["Unnamed: 0", "Unnamed: 0", "link", "hotel_name", "facilities",'image','Free_wifi']

# Drop the specified columns
df.drop(columns_to_drop, axis=1, inplace=True)

# Convert dtype of 'review_score' column from object to float64
df['review_score']= df['review_score'].str.replace(',','.').astype(float)

# Replace NaN values in 'review_score' and 'review_count' columns with mean value
df['review_score'].fillna(df['review_score'].min(), inplace=True)
# df['review_count'].fillna(df['review_count'].min(), inplace=True)
# Convert the 'review_count' column to int
#df['review_count'] = df['review_count'].astype(int)
# Convert 'price' to numerical
df['price'] = df['price'].str.replace('VND', '').str.replace('.', '').str.replace(',', '').astype(int)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2440\1098929373.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  subscore_df = df['subscore'].apply(eval).apply(pd.Series)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2440\1098929373.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  subscore_df = df['subscore'].apply(eval).apply(pd.Series)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2440\1098929373.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  subscore_df = df['subscore'].apply(eval).apply(pd.Series)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2440\1098929373.py:2: FutureWarning: The default dtype for empty Series will be 'object' instea

In [6]:
df

,review_score,review_count,price,address,Pool,No_smoking_room,Families_room,Room_service,Free_parking,Breakfast,24h_front_desk,Airport_shuttle,Service staff,Facilities,Cleanliness,Comfort,Affordable,Location,Free WiFi
0,7.9,29.0,300000,Quận Gò Vấp,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.3,115.0,434700,Quận 9,1,1,1,0,1,0,0,0,9.1,8.1,8.5,8.6,8.5,7.9,8.8
2,4.3,12.0,453739,TP. Hồ Chí Minh,1,1,1,0,1,0,0,0,10.0,10.0,10.0,10.0,10.0,10.0,NaN
3,5.7,278.0,503714,Quận 7,1,1,1,1,1,1,0,0,6.5,6.2,6.3,6.4,6.7,6.4,NaN
4,9.0,373.0,554040,Quận 7,1,1,1,1,1,0,0,1,9.3,9.1,9.2,9.2,9.1,9.3,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,8.6,40.0,848000,Quận 1,1,0,1,1,1,0,0,1,9.5,8.4,8.8,8.9,8.8,9.5,10.0
477,8.1,24.0,850000,Quận 3,1,1,1,1,1,0,0,1,8.4,8.2,8.2,8.6,8.8,9.1,5.0
478,9.3,4.0,877500,Quận Phú Nhuận,1,0,1,1,0,1,0,1,10.0,9.4,8.8,9.4,9.2,9.4,NaN
479,9.5,401.0,880000,Quận 4,1,1,1,1,1,0,0,1,9.7,9.5,9.5,9.6,9.5,9.6,9.9
